In [2]:
import torch
from torch.utils.data import DataLoader
from datasets.cfg import num_classes, max_size
from models.models import build_model
from datasets.loader import DataModule, DataConfig
from datasets.base import collate_bb
from train.eval import Evaluator
from train.unsupervised_metrics import UnlabeledSanity

device = torch.device("cuda")

model_B = build_model("maskrcnn_r50_fpn", num_classes).to(device)
model_B.load_state_dict(torch.load("./weights/maskrcnn_B_ep40.pth", map_location="cuda"))

model_AB = build_model("maskrcnn_r50_fpn", num_classes).to(device)
model_AB.load_state_dict(torch.load("./weights/maskrcnn_AB_ep40.pth", map_location="cuda"))

data = DataModule(DataConfig())
loader_B_test = data.make_loader_b_test()
loader_A_val  = data.make_loader_a_val()
loader_C_test  = data.make_loader_c_test()

/tmp/ipykernel_3163150/3733450509.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_B.load_state_dict(torch.load("./weights/maskrcnn_B_ep40.pth", map_location="cuda

In [3]:
e = Evaluator(device)
mb_B  = e.metrics_masks(model_B,  loader_B_test, num_classes=num_classes - 1)
mb_AB = e.metrics_masks(model_AB, loader_B_test, num_classes=num_classes - 1)

rows = [
    ["setup", "mAP50", "PQ_all", "mPQ", "AJI"],
    ["Model B", f"{mb_B['mAP50']:.3f}",  f"{mb_B['PQ_all']:.3f}",  f"{mb_B['mPQ']:.3f}",  f"{mb_B['AJI']:.3f}"],
    ["Model A+B",   f"{mb_AB['mAP50']:.3f}", f"{mb_AB['PQ_all']:.3f}", f"{mb_AB['mPQ']:.3f}", f"{mb_AB['AJI']:.3f}"],
]
print("\n=== B-test metrics ===")
e.print_table(rows)

map50_A_val = e.map50(model_AB, loader_A_val)
print(f"\nA-val mAP50 (boxes, A+B model): {map50_A_val:.3f}")

map50_C = e.map50(model_AB, loader_C_test)
print(f"\nChromo-CRCN test mAP50 (boxes, A+B model): {map50_C:.3f}")


=== B-test metrics ===
setup      mAP50  PQ_all  mPQ    AJI  
Model B    0.885  0.582   0.581  0.504
Model A+B  0.905  0.652   0.651  0.599

A-val mAP50 (boxes, A+B model): 0.964

Chromo-CRCN test mAP50 (boxes, A+B model): -1.000


In [9]:
sanity = UnlabeledSanity(num_classes=24)
san_scores_ab_b = e.sanity(model_AB, loader_B_test, sanity)
san_scores_b_b = e.sanity(model_B, loader_B_test, sanity)
san_scored_ab_d = e.sanity(model_AB, loader_D_test, sanity)
ds = san_scores_b_b["dataset_scores"]

r0 = san_scores_b_b["per_image"][0] #sanity for one image
print("B model: image 0 -> n:", r0.n_instances, "sex:", r0.sex_pattern)

r0 = san_scores_ab_b["per_image"][0] #sanity for one image
print("AB model: image 0 -> n:", r0.n_instances, "sex:", r0.sex_pattern)

print("\n=== Unlabeled B-test sanity (B model) ===")
print("total:", f"{ds['total']:.3f}",
      "count:", f"{ds['count']:.3f}",
      "balance:", f"{ds['class_balance']:.3f}",
      "sex:", f"{ds['sex']:.3f}",
      "size:", f"{ds['size']:.3f}")

ds = san_scores_ab_b["dataset_scores"]
print("\n=== Unlabeled B-test sanity (A+B model) ===")
print("total:", f"{ds['total']:.3f}",
      "count:", f"{ds['count']:.3f}",
      "balance:", f"{ds['class_balance']:.3f}",
      "sex:", f"{ds['sex']:.3f}",
      "size:", f"{ds['size']:.3f}")

ds = san_scores_ab_d["dataset_scores"]
print("\n=== Unlabeled D-test sanity (A+B model) ===")
print("total:", f"{ds['total']:.3f}",
      "count:", f"{ds['count']:.3f}",
      "balance:", f"{ds['class_balance']:.3f}",
      "sex:", f"{ds['sex']:.3f}",
      "size:", f"{ds['size']:.3f}")



KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/neoph/dev/Train/.tenv312/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/neoph/dev/Train/.tenv312/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/home/neoph/dev/Train/Jupyter/KaryoTest/datasets/dataset_a.py", line 30, in __getitem__
    lab = self.label_map[name]
          ~~~~~~~~~~~~~~^^^^^^
KeyError: 'chromosomes'


In [8]:
loader_D_test  = data.make_loader_d_test()

In [10]:
e.show_examples(data.ds_b_test, model_B,  n=1, score_thresh=0.5, title="B-test, B model")
e.show_examples(data.ds_b_test, model_AB, n=1, score_thresh=0.5, title="B-test, A+B model")


Examples: B-test, B model


TypeError: plot_predictions() got an unexpected keyword argument 'random'